In [1]:
import argparse
import os
import pickle
import sys
from collections import Counter
from datetime import datetime

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from tensorflow import keras
# from tensorflow.keras.layers import (Activation, Conv2D, Dense, Dropout, Flatten,
#                           MaxPooling2D)
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.utils import Sequence
from CNN_singleImage_baseline import DataGenerator

In [2]:
print(tf.__version__)
print("GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
GPUs Available:  1


In [3]:
# load the model
modelpath = '/data/models/'
os.listdir(modelpath)

['train.pickle',
 'simpleCNN_1000epochs_20191110_153647.h5',
 'simpleCNN_1000epochs_20191110_171702.h5',
 'test.pickle',
 'simpleCNN_history_10epochs_20191109_192747.pickle',
 'simpleCNN_10epochs_20191109_192747.h5',
 'simpleCNN_history_1000epochs_20191110_150328.pickle',
 'simpleCNN_100epochs_20191110_033120.h5',
 'simpleCNN_history_1000epochs_20191110_153647.pickle',
 'simpleCNN_history_1000epochs_20191110_171702.pickle',
 'simpleCNN_history_100epochs_20191110_033120.pickle',
 'simpleCNN_1000epochs_20191110_150328.h5',
 'predictions.pickle']

In [4]:
modelfile = 'simpleCNN_1000epochs_20191110_171702.h5'
model = keras.models.load_model(os.path.join(modelpath, modelfile))

In [5]:
with open(os.path.join(modelpath, 'test.pickle'), 'rb') as f:
    x_test, y_test = pickle.load(f)
print(len(x_test), len(y_test))

346 346


In [6]:
batch_size = 1
test_generator = DataGenerator(x_test, y_test, batch_size)

In [7]:
y_pred = model.predict_generator(test_generator)

In [8]:
y_pred.shape

(346, 5)

In [9]:
np.sum(y_pred > 0.5, axis=0)

array([325,  10,   0,   3,   0])

In [10]:
import copy
y_prob = copy.copy(y_pred)
y_pred = (y_pred > 0.5).astype(np.int)
with open(os.path.join(modelpath, 'predictions.pickle'), 'wb') as f:
    pickle.dump((y_prob, y_pred), f, protocol=-1)

In [11]:
labelPath = 'week10_labeled.csv'
df = pd.read_csv(labelPath)

In [12]:
new_files = []
for file in df.clip_title:
    newfile = ''.join(file.split('.mp4')) + '.mp4'
    new_files.append(newfile)
df['clip_title'] = new_files
df['label'] = df['class']
df.drop(columns=['class'], inplace=True)
df.groupby('label').size()

label
-1       21
 0     1259
 1      319
 2       14
 3       84
 4       52
 99    3692
dtype: int64

In [13]:
df = df.loc[df.label != -1]
df = df.loc[df.label != 99]
df.groupby('label').size()

label
0    1259
1     319
2      14
3      84
4      52
dtype: int64

In [14]:
df.shape

(1728, 4)

In [19]:
# vidPath = '/data/vids/scaled'
filenames = [f.split('.mp4')[0] + '_scaled.mp4' for f in df.clip_title]
# filenames = [os.path.join(vidPath, f) for f in filenames]
# labels = df.punch.tolist()
labels = df.label.tolist()


In [20]:
df['clip_title'] = filenames

In [21]:
df2 = pd.DataFrame()

In [22]:
filenames2 = [x.split('/')[-1] for x in x_test]
df2['clip_title'] = filenames2

In [23]:
df2.merge(df, how='inner', on='clip_title')

,clip_title,video-UID,labeler,label
0,V_789_slice50_scaled.mp4,12444,Jen,1
1,NV_648_scaled.mp4,10937,Lance,0
2,heavy_bag_drills_slice20_scaled.mp4,3290,Jen,0
3,streetfight2_slice280_scaled.mp4,14726,Alex,0
4,hook_punch_power_slice65_scaled.mp4,3425,Alex,0
...,...,...,...,...
341,how_to_punch_7_basics_slice90_scaled.mp4,3492,Jen,0
342,how_to_punch_hard_fast_slice195_scaled.mp4,3516,Ahsen,4
343,Australian_flash_mob_dance_slice95_scaled.mp4,10064,Ahsen,0
344,short_strikes_slice145_scaled.mp4,14251,Alex,0
